In [52]:
import pandas as pd
import numpy as np

In [53]:
train = pd.read_csv("./data/7. train 이상치수정완료.csv",encoding='cp949',index_col=0)
train.head()
test1 = pd.read_csv("./data/6. test1.csv",index_col=0)
test2 = pd.read_csv("./data/6. test2.csv",index_col=0)

In [54]:
col = train.columns[7:55]
train_1 = train[col]
test_1 = test1[col]
test_2 = test2[col]

In [55]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(train_1)

train_Min = scaler.transform(train_1)
test1_Min = scaler.transform(test_1)
test2_Min = scaler.transform(test_2)
train_Min = pd.DataFrame(train_Min,columns = train_1.columns)

In [56]:
train[col] = train_Min
test1[col] = test1_Min
test2[col] = test2_Min

# 1. MDA

In [57]:
# 1-1 정규성 테스트(샤피로)
from scipy.stats import norm
from scipy import stats
from statsmodels.formula.api import ols
from scipy.stats import kstest

for j in col:
    a = stats.shapiro(train_Min[j])
    p = a.pvalue
    print(j,a)

매출액증가율 ShapiroResult(statistic=0.8382039070129395, pvalue=6.445972935894159e-44)
유형자산증가율 ShapiroResult(statistic=0.5012198686599731, pvalue=0.0)
재고자산증가율 ShapiroResult(statistic=0.5221829414367676, pvalue=0.0)
비유동자산증가율 ShapiroResult(statistic=0.733120322227478, pvalue=0.0)
유동자산증가율 ShapiroResult(statistic=0.8816322088241577, pvalue=3.204322573700737e-39)
자기자본증가율 ShapiroResult(statistic=0.8314108848571777, pvalue=1.5414283107572988e-44)
총자본증가율 ShapiroResult(statistic=0.8660170435905457, pvalue=4.744656470357398e-41)
순이익증가율 ShapiroResult(statistic=0.5167244672775269, pvalue=0.0)
매출액총이익률 ShapiroResult(statistic=0.9277033805847168, pvalue=2.2633388469962743e-32)
매출액순이익률 ShapiroResult(statistic=0.4788692593574524, pvalue=0.0)
자기자본순이익률 ShapiroResult(statistic=0.6980950832366943, pvalue=0.0)
총자본사업이익률 ShapiroResult(statistic=0.9500693678855896, pvalue=1.1102103090512913e-27)
총자본순이익률 ShapiroResult(statistic=0.8355908393859863, pvalue=3.6433760072445244e-44)
1주당매출액 ShapiroResult(statistic=0.446581

In [58]:
# 1-2 정규성 테스트(K-S)
for j in col:
    a = kstest(train_Min[j],'norm')
    p = a.pvalue
    print(j,a)


매출액증가율 KstestResult(statistic=0.1494450368982022, pvalue=1.1362396859850882e-46)
유형자산증가율 KstestResult(statistic=0.25534676708817095, pvalue=1.0024992696463595e-136)
재고자산증가율 KstestResult(statistic=0.2487981308708953, pvalue=1.0022857955124997e-129)
비유동자산증가율 KstestResult(statistic=0.18961399478604568, pvalue=4.3392196534424174e-75)
유동자산증가율 KstestResult(statistic=0.11769236630380164, pvalue=4.6452063130117473e-29)
자기자본증가율 KstestResult(statistic=0.1501156702092289, pvalue=4.340682713207459e-47)
총자본증가율 KstestResult(statistic=0.12414098042970467, pvalue=2.716422617677006e-32)
순이익증가율 KstestResult(statistic=0.2944647299929928, pvalue=1.3362989085493004e-182)
매출액총이익률 KstestResult(statistic=0.10390739656016212, pvalue=9.853691269214505e-23)
매출액순이익률 KstestResult(statistic=0.28982740255376677, pvalue=8.547734147398325e-177)
자기자본순이익률 KstestResult(statistic=0.20994555000454307, pvalue=4.004827714090454e-92)
총자본사업이익률 KstestResult(statistic=0.10326520786110222, pvalue=1.8576185087406493e-22)
총자본순이익률 K

In [59]:
train_Min["label"] = train["label"]

#### p-value가 0 이는 데이터개수가 많아서 p-value 자체가 너무작아 계산이 불가능하다고 판단. 중심극한 정리에 의해서 정규성이 있다고 가정하고 진행

In [60]:
# 부도기업과 정상기업의 피처별 등분산비교 (정규성 가정으로 bartlett)
Bad = train_Min[train_Min['label']== 1] #Existing Customer
Good = train_Min[train_Min['label']== 0] #Attrited Customer

c = []
for i in train_Min:
    # lresult = stats.levene(close[i], normal[i])
    lresult = stats.bartlett(Bad[i], Good[i])
    c.append([i,lresult[-1]])

c= pd.DataFrame(c)
c.columns=["피처값",'F-test']
c

c:\Users\OPERATOR-70\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\stats\_morestats.py:2507: RuntimeWarning: divide by zero encountered in log
  numer = (Ntot*1.0 - k) * log(spsq) - np.sum((Ni - 1.0)*log(ssq), axis=0)
c:\Users\OPERATOR-70\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\stats\_morestats.py:2507: RuntimeWarning: invalid value encountered in double_scalars
  numer = (Ntot*1.0 - k) * log(spsq) - np.sum((Ni - 1.0)*log(ssq), axis=0)


,피처값,F-test
0,매출액증가율,1.417322e-06
1,유형자산증가율,9.425294e-10
2,재고자산증가율,1.156878e-02
3,비유동자산증가율,3.089934e-04
4,유동자산증가율,1.768075e-02
5,자기자본증가율,1.764859e-12
6,총자본증가율,2.079932e-01
7,순이익증가율,7.329558e-19
8,매출액총이익률,1.449988e-04
9,매출액순이익률,1.157832e-32


In [61]:
# F-test 결과 0.05 이상이면 homo 0.05 이하이면 hetero
c["분산"] =''
c["T-test"] =""
for i in c.index:
    if c.loc[i,"F-test"]>=0.05:
        c.loc[i,"분산"] = "homo"
    else:
        c.loc[i,"분산"] = "hetero"
c

,피처값,F-test,분산,T-test
0,매출액증가율,1.417322e-06,hetero,
1,유형자산증가율,9.425294e-10,hetero,
2,재고자산증가율,1.156878e-02,hetero,
3,비유동자산증가율,3.089934e-04,hetero,
4,유동자산증가율,1.768075e-02,hetero,
5,자기자본증가율,1.764859e-12,hetero,
6,총자본증가율,2.079932e-01,homo,
7,순이익증가율,7.329558e-19,hetero,
8,매출액총이익률,1.449988e-04,hetero,
9,매출액순이익률,1.157832e-32,hetero,


In [62]:
c[c["분산"]=='homo']

,피처값,F-test,분산,T-test
6,총자본증가율,0.207993,homo,
15,유동비율,0.768681,homo,
16,당좌비율,0.723862,homo,
17,현금비율,0.600805,homo,
26,자기자본회전률,0.630710,homo,
36,순운전자본회전률,0.867793,homo,
39,당좌자산회전률,0.950182,homo,


In [63]:
# homo 인 feature 는 student t-test, hetero이면 Welchs T-Test 진행
c["분산"] =''
c["T-test"] =""
for i in c.index:
    if c.loc[i,"F-test"]>=0.05:
        c.loc[i,"분산"] = "homo"
        result = stats.ttest_ind(Bad[c.loc[i,"피처값"]], Good[c.loc[i,"피처값"]], equal_var=True)       ## equal_var = True Student T-test
        c.loc[i,"T-test"] = result[-1]
    else:
        c.loc[i,"분산"] = "hetero"
        result = stats.ttest_ind(Bad[c.loc[i,"피처값"]], Good[c.loc[i,"피처값"]], equal_var=False)      ## equal_var = False Welchs T-Test
        c.loc[i,"T-test"] = result[-1]
c

C:\Users\OPERATOR-70\AppData\Local\Temp\ipykernel_18880\43181667.py:11: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  result = stats.ttest_ind(Bad[c.loc[i,"피처값"]], Good[c.loc[i,"피처값"]], equal_var=False)      ## equal_var = False Welchs T-Test


,피처값,F-test,분산,T-test
0,매출액증가율,1.417322e-06,hetero,0.001358
1,유형자산증가율,9.425294e-10,hetero,0.558464
2,재고자산증가율,1.156878e-02,hetero,0.015938
3,비유동자산증가율,3.089934e-04,hetero,0.002837
4,유동자산증가율,1.768075e-02,hetero,0.0
5,자기자본증가율,1.764859e-12,hetero,0.000065
6,총자본증가율,2.079932e-01,homo,0.0
7,순이익증가율,7.329558e-19,hetero,0.000022
8,매출액총이익률,1.449988e-04,hetero,0.074929
9,매출액순이익률,1.157832e-32,hetero,0.000003


In [64]:
# 0.05 이상이면 유의미하지 않으므로 0.05 이하인 것만 추출
d = c[c["T-test"]<= 0.05]
d.sort_values('T-test',ascending=False)["피처값"].unique()
# d.sort_values('T-test',ascending=False)["피처값"]

array(['유동자산구성비율', '자본분배율', '이자보상배율이자비용', '매출채권회전기간', '자기자본회전률',
       '재고자산증가율', '현금비율', '당좌비율', '부채비율', '비유동비율', '매입채무회전기간', '당좌자산회전률',
       '유동비율', '비유동자산증가율', '1주당매출액', '설비투자효율', '매출액증가율', '자기자본증가율',
       '부가가치율', '순이익증가율', '자기자본구성비율', '총자본투자효율', '매입채무회전률', '매출액순이익률',
       '자기자본순이익률', '총자본회전률', '1주당순이익', '유동자산증가율', '순운전자본비율', '순운전자본회전률',
       '부가가치', '총자본순이익률', '총자본사업이익률', '비유동자산회전률', '기계투자효율', '자기자본배율',
       '총자본증가율', 'label'], dtype=object)

In [65]:
d.sort_values('T-test',ascending=False).dropna()

,피처값,F-test,분산,T-test
22,유동자산구성비율,3.318390e-02,hetero,0.03795
46,자본분배율,3.334576e-11,hetero,0.032797
20,이자보상배율이자비용,1.537723e-03,hetero,0.031688
33,매출채권회전기간,5.345712e-13,hetero,0.030651
26,자기자본회전률,6.307105e-01,homo,0.019695
2,재고자산증가율,1.156878e-02,hetero,0.015938
17,현금비율,6.008051e-01,homo,0.013937
16,당좌비율,7.238616e-01,homo,0.013597
19,부채비율,3.098096e-32,hetero,0.008211
21,비유동비율,1.651442e-35,hetero,0.007159


In [66]:
d.sort_values('T-test',ascending=False).dropna()["피처값"].tolist()

['유동자산구성비율',
 '자본분배율',
 '이자보상배율이자비용',
 '매출채권회전기간',
 '자기자본회전률',
 '재고자산증가율',
 '현금비율',
 '당좌비율',
 '부채비율',
 '비유동비율',
 '매입채무회전기간',
 '당좌자산회전률',
 '유동비율',
 '비유동자산증가율',
 '1주당매출액',
 '설비투자효율',
 '매출액증가율',
 '자기자본증가율',
 '부가가치율',
 '순이익증가율',
 '자기자본구성비율',
 '총자본투자효율',
 '매입채무회전률',
 '매출액순이익률',
 '자기자본순이익률',
 '총자본회전률',
 '1주당순이익',
 '유동자산증가율',
 '순운전자본비율',
 '순운전자본회전률',
 '부가가치',
 '총자본순이익률',
 '총자본사업이익률',
 '비유동자산회전률',
 '기계투자효율',
 '자기자본배율',
 '총자본증가율']

In [67]:
# t-test가 가장 작은순으로 의미 있으므로 가장작은 순위로 10개 피처 선정
fea = ['자기자본구성비율', '매출액순이익률', '순운전자본비율',
       '순운전자본회전률', '총자본투자효율', '자기자본순이익률', '총자본증가율', '총자본사업이익률', '기계투자효율',
       '1주당순이익', '총자본순이익률', '자기자본배율']
mda_feature = train[fea]
mda_feature 

,자기자본구성비율,매출액순이익률,순운전자본비율,순운전자본회전률,총자본투자효율,자기자본순이익률,총자본증가율,총자본사업이익률,기계투자효율,1주당순이익,총자본순이익률,자기자본배율
0,-0.111365,-0.785167,2.084649,-1.209730,-0.985755,-0.831895,4.623407,-1.470744,-0.138332,-0.288793,-1.253677,-0.933622
1,1.440981,0.190508,1.601287,-1.209730,-0.299815,0.112561,-0.387215,-0.115354,-0.136344,-0.216891,0.037012,-0.917301
2,1.285314,0.173028,0.611395,-0.950725,-0.319755,0.098178,-0.119208,-0.285050,-0.137466,-0.218074,0.018008,-0.917292
3,1.368432,0.142438,0.809533,1.768833,1.201962,0.097151,-1.039894,-0.301367,-0.124074,-0.216466,-0.065926,-0.709432
4,1.549563,-0.488692,2.207287,1.139819,-0.839338,-1.298644,-0.209745,-0.613563,-0.136803,-0.482001,-2.793795,-0.809214
...,...,...,...,...,...,...,...,...,...,...,...,...
2365,-0.736916,-0.137477,-0.463185,-0.377213,-0.097565,-0.779844,-0.684051,-1.144406,-0.113658,-0.533590,-0.920315,-0.617170
2366,-2.033660,-0.899709,-1.516194,-0.691719,-1.573704,-4.088179,-1.658789,-2.515025,-0.160161,-1.057579,-3.017884,-0.920813
2367,-0.966572,0.021456,-0.879036,-0.377213,-0.947584,-0.384323,-0.344424,-0.912706,-0.138371,-0.896741,-0.442047,0.660739
2368,0.521873,0.233059,1.326850,2.342345,-0.199544,0.496096,-0.506580,0.799479,-0.109470,0.066265,0.660186,-0.225207


In [68]:
def vif(data):
    import pandas as pd
    from statsmodels.stats.outliers_influence import variance_inflation_factor

    # VIF 출력을 위한 데이터 프레임 형성
    vif = pd.DataFrame()

    # VIF 값과 각 Feature 이름에 대해 설정
    vif["VIF Factor"] = [variance_inflation_factor(data.values, i) for i in range(len(data.columns))]
    vif["features"] = data.columns 
                    
    # VIF 값이 높은 순으로 정렬
    vif = vif.sort_values(by="VIF Factor", ascending=False)
    vif = vif.reset_index().drop(columns='index')
    
    return vif

vif(mda_feature)

,VIF Factor,features
0,9.693618,총자본순이익률
1,3.939593,자기자본순이익률
2,3.394428,총자본사업이익률
3,2.528754,자기자본구성비율
4,2.463932,총자본투자효율
5,2.427343,순운전자본비율
6,2.363377,매출액순이익률
7,1.510243,1주당순이익
8,1.465004,자기자본배율
9,1.342259,순운전자본회전률


# 3. Lasso

In [69]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet

In [70]:
# 12개 피처로 조정 C 값 조정
feature = train_Min.drop('label',axis=1)
target = train['label']
lasso = SelectFromModel(LogisticRegression(C=0.1, penalty='l1',solver='liblinear'))
lasso.fit(feature, target)
lasso_support = lasso.get_support()
lasso_feature = feature.loc[:,lasso_support].columns.tolist()

In [71]:
lasso_feature

['유동자산증가율',
 '총자본증가율',
 '매출액순이익률',
 '총자본사업이익률',
 '총자본순이익률',
 '순운전자본비율',
 '자기자본배율',
 '매출채권회전률',
 '매출채권회전기간',
 '순운전자본회전률',
 '당좌자산회전률']

In [72]:
## 10개 피처 선정 완료
## 성장성'유동자산증가율','총자본증가율',
## 수익성  '자기자본순이익률', '총자본순이익률',
## 생산성 '자본분배율',  '부가가치'
## 안정성 '순운전자본비율', '자기자본배율'
## 활동성 "매입채무회전률",'매출채권회전기간'

In [73]:
fea = [
'유동자산증가율','총자본증가율',
  '자기자본순이익률', '총자본순이익률',
'자본분배율',  '부가가치',
'순운전자본비율', '자기자본배율',
 "매입채무회전률",'매출채권회전기간','label']

lasso_feature_train = train[fea]
lasso_feature_train

,유동자산증가율,총자본증가율,자기자본순이익률,총자본순이익률,자본분배율,부가가치,순운전자본비율,자기자본배율,매입채무회전률,매출채권회전기간,label
0,4.656474,4.623407,-0.831895,-1.253677,-8.418633,-0.577415,2.084649,-0.933622,-0.271193,2.352764,0.0
1,-0.781120,-0.387215,0.112561,0.037012,0.125971,-0.455388,1.601287,-0.917301,-0.293360,1.949635,0.0
2,-0.999905,-0.119208,0.098178,0.018008,0.052882,-0.454182,0.611395,-0.917292,-0.306007,2.344302,0.0
3,-1.231120,-1.039894,0.097151,-0.065926,-0.009179,-0.255864,0.809533,-0.709432,0.116254,-0.863421,0.0
4,1.947777,-0.209745,-1.298644,-2.793795,-8.418633,-0.556276,2.207287,-0.809214,0.266255,-0.976897,0.0
...,...,...,...,...,...,...,...,...,...,...,...
2365,-0.749422,-0.684051,-0.779844,-0.920315,-0.316843,-0.451747,-0.463185,-0.617170,-0.277313,0.586959,0.0
2366,-1.005188,-1.658789,-4.088179,-3.017884,-0.135762,-0.638106,-1.516194,-0.920813,-0.308727,0.855071,0.0
2367,-0.274560,-0.344424,-0.384323,-0.442047,-2.227714,-0.548722,-0.879036,0.660739,-0.301792,0.746210,0.0
2368,-0.845761,-0.506580,0.496096,0.660186,0.536341,-0.264778,1.326850,-0.225207,-0.242770,-0.165446,0.0


In [74]:
lasso_feature_test1 = test1[fea]
lasso_feature_test1

,유동자산증가율,총자본증가율,자기자본순이익률,총자본순이익률,자본분배율,부가가치,순운전자본비율,자기자본배율,매입채무회전률,매출채권회전기간,label
0,-0.416894,-0.411989,-0.173721,-0.508561,-0.537658,-0.504004,0.495548,-0.923950,-0.300704,1.910015,0.0
1,6.016417,4.045050,0.167352,0.158955,0.196599,-0.391450,1.771063,-0.849038,-0.292544,1.203000,0.0
2,-0.097109,-0.350280,0.184816,0.199338,0.191495,-0.333551,1.828987,-0.854769,-0.288600,0.891037,0.0
3,-0.532502,-0.371901,0.140641,-0.329607,-1.020205,-0.533852,1.835778,-0.804897,0.105646,-0.947470,0.0
4,0.538422,3.719387,1.390898,2.117161,0.628203,-0.094689,1.067992,-0.700630,0.049345,-0.911312,0.0
...,...,...,...,...,...,...,...,...,...,...,...
2752,1.073573,0.878052,0.730669,1.523285,0.579903,1.013943,1.048418,13.392970,0.120742,0.024579,0.0
2753,10.648488,7.766982,0.613212,1.151123,0.627839,1.755050,2.460153,29.222842,0.244496,0.372700,0.0
2754,-0.690064,0.287534,0.349531,0.437681,0.420967,-0.303308,1.616068,-0.118177,-0.262897,-0.091783,0.0
2755,0.034348,-0.230465,0.299534,0.371958,0.370389,-0.333219,1.791436,-0.079093,-0.266569,-0.154291,0.0


In [75]:
lasso_feature_test2 = test2[fea]
lasso_feature_test2

,유동자산증가율,총자본증가율,자기자본순이익률,총자본순이익률,자본분배율,부가가치,순운전자본비율,자기자본배율,매입채무회전률,매출채권회전기간,label
0,-0.188476,-0.095335,0.293028,0.425011,0.161968,-0.276622,1.809412,-0.844385,-0.256234,0.838145,0.0
1,2.449365,2.773026,0.160845,0.153412,0.113941,-0.236968,2.200096,-0.780307,-0.189869,0.809103,0.0
2,-0.557985,-0.015157,-0.102493,-0.448382,-2.779796,-0.561153,-0.471174,-0.869480,0.051385,-0.847073,0.0
3,21.459036,13.790178,-0.657934,-0.382660,-0.135762,-0.592425,0.374109,-0.902010,0.244768,15.397528,0.0
4,0.367497,0.316812,0.518013,0.848642,0.250458,0.546981,0.537493,-0.154275,-0.067338,-0.454907,0.0
...,...,...,...,...,...,...,...,...,...,...,...
2198,-0.404464,-0.574145,-1.101055,-1.711358,-0.135762,-0.627497,0.478770,-0.861296,-0.306279,1.233773,0.0
2199,-0.169208,0.241139,0.422814,0.684733,0.599952,1.759931,2.143371,30.247860,0.082663,0.834683,0.0
2200,-0.393276,-0.408836,0.301247,0.435305,0.511006,1.471000,1.958416,28.616352,0.095719,0.153826,0.0
2201,-0.852598,-0.558380,0.276591,0.350579,0.354258,-0.323080,1.687174,0.003890,-0.269697,-0.187757,0.0


In [76]:
lasso_feature_train.to_csv("./data/8-1 train(std).csv")  
lasso_feature_test1.to_csv("./data/8-1 test1(std).csv") 
lasso_feature_test2.to_csv("./data/8-1 test2(std).csv") 